In [4]:
import os

from pyod.models.iforest import IForest
import pandas as pd

from sdk.base import AlgoTemplate

# 为config设置环境变量
# config_file_path = "../config/example/config-iforest-train-example.json"
config_file_path = "../config/example/config-iforest-inference-example.json"
with open(config_file_path) as config_file:
    config_content = config_file.read()
# print("config_content: {}".format(config_content))
# os.environ["MODE"] = "TRAIN"
os.environ["MODE"] = "INFERENCE"

os.environ["CONFIG"] = config_content

class IsolationForest(AlgoTemplate):

    def build_model(self, hyper_params: dict) -> IForest:
        return IForest(verbose=hyper_params["verbose"])

    def train(self, model: IForest, df: pd.DataFrame, hyper_params: dict) -> (object, dict):

        # preprocess
        print("check null: {}".format(df.isnull().values.any()))
        print("df max: {} min:{}".format(df.max(), df.min()))
        min_max = df.apply(lambda x: pd.Series([x.min(), x.max()])).T.values.tolist()
        for i, col in enumerate(df):
            df[col] = (df[col] - min_max[i][0]) / (min_max[i][1] - min_max[i][0])
        print("train df after normalization : {}".format(df))

        # train
        x_train = df.to_numpy()
        model.fit(x_train)
        y_train_pred = model.labels_  # binary labels (0: inliers, 1: outliers)
        y_train_scores = model.decision_scores_  # raw outlier scores
        print(y_train_pred)
        print(y_train_scores)
        return model, {"min_max": min_max}

    def inference(self, model: object, args: dict, df: pd.DataFrame) -> pd.Series:
        # preprocess
        min_max = args["min_max"]
        print("min max for inference: {}".format(min_max))
        for i, col in enumerate(df):
            df[col] = (df[col] - min_max[i][0]) / (min_max[i][1] - min_max[i][0])
        print("inference df after normalization: {}".format(df))
        #inference
        x_test = df.to_numpy()
        y_test_pred = model.predict(x_test)
        y_test_scores = model.decision_function(x_test)
        print(y_test_pred)
        print(y_test_scores)

        result = pd.Series(y_test_pred)
        result.index = df.index
        print(result)
        # return result



model = IsolationForest()

2023-02-25 20:08:56.040 | INFO     | sdk.base:__inference:113 - ..............Starting inference..............
2023-02-25 20:13:29.653 | INFO     | sdk.base:__inference:118 - current message value is:
 {'cmdb_id': 'node3-vm3', 'name': 'record_node_cpu_usage', 'timestamp': '2023-02-25T12:13:29Z', 'value': 1.7916666669770933}
2023-02-25 20:13:29.656 | INFO     | sdk.base:__inference:118 - current message value is:
 {'cmdb_id': 'k8s-large-1665988619', 'name': 'record_node_cpu_usage', 'timestamp': '2023-02-25T12:13:29Z', 'value': 7.0208333347303125}
2023-02-25 20:13:29.658 | INFO     | sdk.base:__inference:118 - current message value is:
 {'cmdb_id': 'kubernetes-master', 'name': 'record_node_cpu_usage', 'timestamp': '2023-02-25T12:13:29Z', 'value': 5.800000000745058}
2023-02-25 20:13:29.658 | INFO     | sdk.base:__inference:118 - current message value is:
 {'cmdb_id': 'node3-vm2', 'name': 'record_node_cpu_usage', 'timestamp': '2023-02-25T12:13:29Z', 'value': 2.116666667473808}
2023-02-25 2

ValueError: cannot reindex from a duplicate axis